In [1]:
# from __future__ import absolute_import, division, print_function, unicode_literals

# from tensorflow.keras.preprocessing import sequence
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Embedding, Dropout, BatchNormalization
# from tensorflow.keras.datasets import imdb
# from keras.datasets import boston_housing
import numpy as np
import tensorflow as tf
from tensorflow import keras
from progressbar import ProgressBar

import os
import sys
sys.path.append('..')  
from GradPri_utils.utils import *
from tensorflow.keras.preprocessing import image

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

In [2]:
# DATA_PATH_train = 'cats_dogs/train'
# file_name_train_cats = getfile_name(os.path.join(DATA_PATH_train, 'cats'))
# file_name_train_dogs = getfile_name(os.path.join(DATA_PATH_train, 'dogs'))
# input_size = 224 #224 for Vgg16, Vgg19 and ResNet101V2; 150 for Xception; 299 for InceptionV3

# trainX = []
# trainY = []
# pbar = ProgressBar()
# for i in pbar(range(len(file_name_train_cats))):
#     img_path = os.path.join(DATA_PATH_train, 'cats', file_name_train_cats[i])
#     img = image.load_img(img_path, target_size=(input_size, input_size))
#     x_tmp = image.img_to_array(img)
#     trainX.append(x_tmp)
#     trainY.append(0)

# pbar = ProgressBar()
# for i in pbar(range(len(file_name_train_dogs))):
#     img_path = os.path.join(DATA_PATH_train, 'dogs', file_name_train_dogs[i])
#     img = image.load_img(img_path, target_size=(input_size, input_size))
#     x_tmp = image.img_to_array(img)
#     trainX.append(x_tmp)
#     trainY.append(1)

100% |########################################################################|
100% |########################################################################|


In [3]:
# DATA_PATH_test = 'cats_dogs/test'
# file_name_test_cats = getfile_name(os.path.join(DATA_PATH_test, 'cats'))
# file_name_test_dogs = getfile_name(os.path.join(DATA_PATH_test, 'dogs'))

# testX = []
# testY = []
# pbar = ProgressBar()
# for i in pbar(range(len(file_name_test_cats))):
#     img_path = os.path.join(DATA_PATH_test, 'cats', file_name_test_cats[i])
#     img = image.load_img(img_path, target_size=(input_size, input_size))
#     x_tmp = image.img_to_array(img)
#     testX.append(x_tmp)
#     testY.append(0)

# pbar = ProgressBar()
# for i in pbar(range(len(file_name_test_dogs))):
#     img_path = os.path.join(DATA_PATH_test, 'dogs', file_name_test_dogs[i])
#     img = image.load_img(img_path, target_size=(input_size, input_size))
#     x_tmp = image.img_to_array(img)
#     testX.append(x_tmp)
#     testY.append(1)

100% |########################################################################|
100% |########################################################################|


In [4]:
# trainX = np.array(trainX)
# trainY = np.array(trainY)
# testX = np.array(testX)
# testY = np.array(testY)

In [10]:
# np.save('cats_dogs/trainX.npy', trainX)
# np.save('cats_dogs/trainY.npy', trainY)
# np.save('cats_dogs/testX.npy', testX)
# np.save('cats_dogs/testY.npy', testY)

In [2]:
input_size = 224 #224 for Vgg16, Vgg19 and ResNet101V2; 150 for Xception; 299 for InceptionV3

trainX = np.load('cats_dogs/trainX.npy')
trainY = np.load('cats_dogs/trainY.npy')
testX = np.load('cats_dogs/testX.npy')
testY = np.load('cats_dogs/testY.npy')

In [3]:
print(np.shape(trainX), np.shape(trainY), np.shape(testX), np.shape(testY))

(20000, 224, 224, 3) (20000,) (5000, 224, 224, 3) (5000,)


In [6]:
# # First, instantiate a base model with pre-trained weights.
# base_model = keras.applications.Xception(
#     weights='imagenet',  # Load weights pre-trained on ImageNet.
#     input_shape=(input_size, input_size, 3),
#     include_top=False)  # Do not include the ImageNet classifier at the top.

# base_model = keras.applications.InceptionV3(
#     weights='imagenet',  # Load weights pre-trained on ImageNet.
#     input_shape=(input_size, input_size, 3),
#     include_top=False)  # Do not include the ImageNet classifier at the top.

# base_model = keras.applications.ResNet101V2(
#     weights='imagenet',  # Load weights pre-trained on ImageNet.
#     input_shape=(input_size, input_size, 3),
#     include_top=False)  # Do not include the ImageNet classifier at the top.

base_model = keras.applications.VGG19(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(input_size, input_size, 3),
    include_top=False)  # Do not include the ImageNet classifier at the top.

# base_model = keras.applications.VGG16(
#     weights='imagenet',  # Load weights pre-trained on ImageNet.
#     input_shape=(input_size, input_size, 3),
#     include_top=False)  # Do not include the ImageNet classifier at the top.

In [7]:
# Then, freeze the base model.
base_model.trainable = False

In [8]:
# Create a new model on top.
inputs = keras.Input(shape=(input_size, input_size, 3))
# We make sure that the base_model is running in inference mode here,
# by passing `training=False`. This is important for fine-tuning, as you will
# learn in a few paragraphs.
x = base_model(inputs, training=False)
# Convert features of shape `base_model.output_shape[1:]` to vectors

x = keras.layers.GlobalAveragePooling2D()(x)

# x = keras.layers.Conv2D(1024, (3, 3), activation='relu', padding='same')(x)
# x = keras.layers.Conv2D(1024, (3, 3), activation='relu', padding='same')(x)
# x = keras.layers.MaxPooling2D((2, 2), strides=(2, 2))(x)
# x = keras.layers.Flatten()(x)
# x = keras.layers.Dense(1024, activation='relu')(x)
# x = keras.layers.Dense(128, activation='relu')(x)

# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
vgg19 (Functional)           (None, 7, 7, 512)         20024384  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 513       
Total params: 20,024,897
Trainable params: 513
Non-trainable params: 20,024,384
_________________________________________________________________


In [9]:
# Train the model on new data.
model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[keras.metrics.BinaryAccuracy()])

model.fit(x=trainX, y=trainY, batch_size=128, epochs=10, shuffle=True)

Epoch 1/10
157/157 [==============================] - 52s 251ms/step - loss: 0.7829 - binary_accuracy: 0.8124
Epoch 2/10
157/157 [==============================] - 41s 264ms/step - loss: 0.1704 - binary_accuracy: 0.9433
Epoch 3/10
157/157 [==============================] - 42s 264ms/step - loss: 0.1273 - binary_accuracy: 0.9556
Epoch 4/10
157/157 [==============================] - 38s 241ms/step - loss: 0.1038 - binary_accuracy: 0.9634
Epoch 5/10
157/157 [==============================] - 32s 206ms/step - loss: 0.0917 - binary_accuracy: 0.9665
Epoch 6/10
157/157 [==============================] - 29s 186ms/step - loss: 0.0818 - binary_accuracy: 0.9686
Epoch 7/10
157/157 [==============================] - 27s 169ms/step - loss: 0.0757 - binary_accuracy: 0.9704
Epoch 8/10
157/157 [==============================] - 26s 165ms/step - loss: 0.0703 - binary_accuracy: 0.9724
Epoch 9/10
157/157 [==============================] - 26s 162ms/step - loss: 0.0666 - binary_accuracy: 0.9739
Epoch 10/1

In [13]:
model.evaluate(x=trainX, y=trainY, batch_size=128)

In [12]:
model.evaluate(x=testX, y=testY, batch_size=128)

40/40 [==============================] - 7s 147ms/step - loss: 0.0720 - binary_accuracy: 0.9718


[0.07199510186910629, 0.9718000292778015]